# import

In [24]:
from gensim.models import Word2Vec
import numpy as np
import pandas as pd

user = pd.read_csv('./clean_df.csv', index_col=0)
problem = pd.read_excel('./data/problemList.xlsx')

In [18]:
import ast
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import torch

from torch.utils.data import DataLoader, TensorDataset
import gc
import torch.nn as nn
from collections import Counter



device = torch.device('mps')

# kmeans

In [25]:
user['solvedProblemList'] = user['solvedProblemList'].apply(ast.literal_eval)
user['triedNotsolvedList'] = user['triedNotsolvedList'].apply(ast.literal_eval)

In [5]:
def list_to_text(lst):
    return ' '.join(map(str, lst))

user['solvedProblemList_text'] = user['solvedProblemList'].apply(list_to_text)

In [7]:
model = Word2Vec(sentences=user['solvedProblemList_text'], vector_size=100, window=2, min_count=1000, workers=4)

In [8]:
# Word2Vec 모델을 사용하여 각 리스트의 벡터 평균을 계산하여 새로운 특성 생성
def average_vector(problem_list):
    # 리스트 내의 모든 문제에 대해 벡터를 구하고 평균을 계산
    vectors = [model.wv[word] for word in problem_list if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)


# 새로운 임베딩 특성을 데이터프레임에 추가
user['solvedProblemListVec'] = user['solvedProblemList_text'].apply(average_vector)

In [19]:
X = user[['rank', 'rightCnt', 'wrongCnt', 'time out', 'memory exceed', 'print exceed', 'runTime error', 'compile error', 'solvedProblemListVec', 'tier']]


In [21]:
# Word2Vec 벡터를 포함한 독립변수 데이터를 합침
X_expanded = X.drop('solvedProblemListVec', axis=1)
X_expanded = X_expanded.join(pd.DataFrame(X['solvedProblemListVec'].tolist()))

In [24]:
# NA 값을 가진 행을 찾음
na_rows = X_expanded.isna().any(axis=1)

# X_expanded에서 NA 값을 가진 행을 제거
X_expanded.dropna(inplace=True)

# 모든 열 이름을 문자열로 변환
X_expanded.columns = X_expanded.columns.astype(str)

In [25]:
# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_expanded)

In [39]:
kmeans = KMeans(n_clusters=100, random_state=0).fit(X_scaled)

/Users/yunju/miniforge3/envs/tave/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [40]:
labels = kmeans.labels_


In [41]:
X_expanded['new_tier'] = labels

In [42]:
tier_counts = X_expanded.groupby('tier')['new_tier'].value_counts()


In [43]:
tier_counts[1]
# 진짜 너무 별론디... 걍 문제번호를 드롭하고 해보자

new_tier
3     2
61    2
50    2
15    1
57    1
44    1
41    1
39    1
58    1
75    1
95    1
93    1
80    1
83    1
84    1
21    1
29    1
35    1
36    1
2     1
Name: count, dtype: int64

In [5]:
X = user[['rank', 'rightCnt', 'wrongCnt', 'tier']]
# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
kmeans = KMeans(n_clusters=50, random_state=0).fit(X_scaled)
labels = kmeans.labels_

X['new_tier'] = labels


/Users/yunju/miniforge3/envs/tave/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/var/folders/d6/d15px6nn599bb66tcmrq23380000gn/T/ipykernel_10196/2804835096.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['new_tier'] = labels


In [6]:
tier_counts = X.groupby('tier')['new_tier'].value_counts()
tier_counts[1]

new_tier
48    23
Name: count, dtype: int64

# knn

In [7]:
user['new_tier'] = labels

In [8]:
# 필요한 칼럼만 선택
x = user[['rank', 'rightCnt', 'wrongCnt', 'tier']]
y = user['new_tier']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# KNN 모델 훈련
knn = KNeighborsClassifier(n_neighbors=20)

knn.fit(X_scaled, y_train)


KNeighborsClassifier(n_neighbors=20)

In [9]:
# 모델 평가
y_pred = knn.predict(X_test_scaled)
report = classification_report(y_test, y_pred)


/Users/yunju/miniforge3/envs/tave/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yunju/miniforge3/envs/tave/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yunju/miniforge3/envs/tave/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [10]:
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1207
           1       0.96      0.99      0.97       290
           2       1.00      1.00      1.00       877
           3       0.95      1.00      0.97        37
           4       0.93      0.93      0.93       152
           5       0.99      1.00      1.00       635
           6       1.00      0.33      0.50         6
           7       0.93      0.90      0.92       144
           8       1.00      1.00      1.00      1052
           9       0.78      0.93      0.85        15
          10       0.99      1.00      1.00      1261
          12       0.97      0.99      0.98       631
          13       1.00      0.99      0.99       761
          14       0.95      0.86      0.90        22
          15       0.89      0.91      0.90        56
          16       0.96      0.99      0.98       221
          17       1.00      1.00      1.00       743
          18       0.92    

# 가장 가까운 데이터 포인터 찾기

In [96]:
distances, indices = knn.kneighbors([X_test_scaled[0]])

In [129]:
# 필터링된 데이터 프레임 생성
nearest_neighbors_df = X_train.iloc[indices[0]]

In [131]:
# 필터링된 데이터 프레임에서 solvedProblemList 칼럼 추출
filtered_solved_problems = user.loc[nearest_neighbors_df.index, 'solvedProblemList']

# 모든 이웃의 solvedProblemList를 하나의 집합으로 합치기
neighbor_solved_problems_set = set().union(*filtered_solved_problems)



In [121]:
len(neighbor_solved_problems_set)

877

In [133]:
# X_test의 특정 사용자의 solvedProblemList 추출 (예시로 첫 번째 사용자를 사용)
test_user_solved_problems = set(user.iloc[X_test.index[0]]['solvedProblemList'])

# 차집합 구하기 (이웃이 풀었지만 테스트 사용자가 풀지 않은 문제들)
unsolved_problems = neighbor_solved_problems_set - test_user_solved_problems


In [114]:
len(unsolved_problems)

733

In [126]:
# 너무 문제가 많으니 가장 가까운 애만 확인해보자

# 필터링된 데이터 프레임에서 solvedProblemList 칼럼 추출
filtered_solved_problems = user.loc[indices[0][0], 'solvedProblemList']

# 모든 이웃의 solvedProblemList를 하나의 집합으로 합치기
neighbor_solved_problems_set = set().union(*filtered_solved_problems)


In [136]:
filtered_solved_problems = user.loc[indices[0][0], 'solvedProblemList']


In [139]:
unsolved_problems = set(filtered_solved_problems) - test_user_solved_problems


In [142]:
len(unsolved_problems)

252

In [ ]:
# 이래도 겁나 많네...... 이걸 이제 ease 모델에 넣어보자

# ease

In [11]:


gc.collect()
torch.mps.empty_cache()


class EASE(nn.Module):
    def __init__(self, num_items, reg_lambda=500):
        super(EASE, self).__init__()
        self.reg_lambda = reg_lambda
        self.num_items = num_items
        self.item_weights = nn.Parameter(torch.zeros(num_items, num_items))
        self.device = torch.device('mps')

    def forward(self, x):
        # x: 사용자-아이템 상호작용 행렬
        return torch.matmul(x, self.item_weights)
    def clear_memory(self):
        gc.collect()
        torch.cuda.empty_cache()
    
    def fit(self, data_loader):
        self.item_weights.data.zero_()  # 가중치 초기화

        for batch in data_loader:
            # 데이터 텐서 추출
            data_tensor = batch[0].to(self.device)

            # 배치에 대한 Gram Matrix 계산
            G_batch = torch.matmul(data_tensor.T, data_tensor) + self.reg_lambda * torch.eye(self.num_items, device=self.device)

            # Batch Gram Matrix의 역행렬 계산
            P_batch = torch.inverse(G_batch)

            # Batch Item Weights 계산
            B_batch = P_batch / -torch.diag(P_batch)
            B_batch.fill_diagonal_(0)

            # 각 배치의 결과를 누적
            self.item_weights.data += B_batch

        # 평균 가중치 계산
        self.item_weights.data /= len(data_loader)


In [12]:
user[user['id'] == 'jh5154']
# 이데이터 찾아야됨

,id,tier,rank,rightCnt,wrongCnt,time out,memory exceed,print exceed,runTime error,compile error,solvedProblemList,triedNotsolvedList,new_tier
0,jh5154,15,10033,546,252,76,2,7,42,33,"[1000, 1001, 1003, 1008, 1012, 1018, 1021, 102...","[1722, 1725, 1916, 11279, 12902, 13199, 13398]",1


In [13]:
# X_train에서 원본 인덱스 위치 찾기
original_index = X_train.index.get_loc(0)

# X_scaled에서 해당 인덱스 위치의 데이터 얻기
scaled_data = X_scaled[original_index]

In [26]:
distances, indices = knn.kneighbors([scaled_data])

# 필터링된 데이터 프레임 생성
nearest_neighbors_df = X_train.iloc[indices[0]]

df = user.loc[nearest_neighbors_df.index, :]

In [28]:
# 문제 번호의 빈도를 계산하기 위한 Counter 객체 초기화
problem_counts = Counter()

# solvedProblemList 칼럼에서 문제 번호의 빈도를 계산
for problem_list in df['solvedProblemList']:
    problem_counts.update(problem_list)


# triedNotsolvedList 칼럼에서 문제 번호의 빈도를 계산
for problem_list in df['triedNotsolvedList']:
    problem_counts.update(problem_list)

In [32]:


# problem_counts에서 값이 5이하면 버리기 => 20명중에서 5명이상이 풀었으면 잘  풀은거지
filtered_problems = {problem: count for problem, count in problem_counts.items() if count >= 5}

# 필터링된 문제 번호 집합 생성
filtered_problems_set = set(filtered_problems.keys())

# solvedProblemList와 triedNotsolvedList에서 필터링된 문제 번호만 유지
df['solvedProblemList'] = df['solvedProblemList'].apply(lambda x: [problem for problem in x if problem in filtered_problems_set])
df['triedNotsolvedList'] = df['triedNotsolvedList'].apply(lambda x: [problem for problem in x if problem in filtered_problems_set])


In [36]:


# 모든 문제의 고유 목록 생성
all_problems = set()
for row in df.itertuples():
    all_problems.update(row.solvedProblemList)
    all_problems.update(row.triedNotsolvedList)
all_problems = sorted(all_problems)

# 이진 상호작용 행렬 생성
interaction_matrix = pd.DataFrame(0, index=df['id'], columns=all_problems)

for row in df.itertuples():
    interaction_matrix.loc[row.id, row.solvedProblemList] = 1




In [38]:
# 데이터프레임을 PyTorch 텐서로 변환
train_tensor = torch.tensor(interaction_matrix.values).float().to(device)


In [39]:
# TensorDataset 및 DataLoader 생성
dataset = TensorDataset(train_tensor)
data_loader = DataLoader(dataset, batch_size=5000, shuffle=True)

model = EASE(num_items=train_tensor.shape[1], reg_lambda=100).to(device)

model.fit(data_loader)


In [40]:
# 문제 번호와 칼럼 인덱스의 매핑 생성
problem_to_index = {problem: idx for idx, problem in enumerate(interaction_matrix.columns)}
solved_problems = user[user['id'] == 'jh5154']['solvedProblemList'].to_list()[0]

# 사용자가 풀었던 문제 번호를 인덱스로 변환
solved_problems_indices = [problem_to_index[problem] for problem in solved_problems if problem in problem_to_index]
# 전체 문제 수
num_problems = train_tensor.shape[1]

In [41]:


# 사용자 상호작용 텐서 생성
user_interaction_tensor = torch.zeros(num_problems)
user_interaction_tensor[solved_problems_indices] = 1
# 텐서를 모델이 있는 디바이스로 이동
user_interaction_tensor = user_interaction_tensor.to(device)
# 모델을 사용하여 추천 점수 계산
predicted_scores = model(user_interaction_tensor)

# 이미 풀었던 문제의 점수를 제외
predicted_scores[solved_problems_indices] = -np.inf

# 추천 점수 기반으로 추천 문제 선택 (예: 상위 N개 문제)
recommended_problems = torch.topk(predicted_scores, k=10).indices
# 칼럼 인덱스에서 문제 번호로의 역매핑 생성 -> 위에있는 값이 value라서 key로 찾을 수 없음 그래서 역매핑해야됨
index_to_problem = {idx: problem for problem, idx in problem_to_index.items()}
# 추천된 인덱스를 문제 번호로 변환
recommended_problem_numbers = [index_to_problem[idx.item()] for idx in recommended_problems]
recommended_problem_numbers =list(map(int, recommended_problem_numbers))
problem[problem['problemId'].isin(recommended_problem_numbers)]

,problemId,title,acceptedUserCount,level,key,bojTagId
149,1149,RGB거리,44529,10,['dp'],[25]
537,1541,잃어버린 괄호,34181,9,"['greedy', 'math', 'parsing', 'string']","[33, 124, 96, 158]"
923,1931,회의실 배정,44542,10,"['greedy', 'sorting']","[33, 97]"
924,1932,정수 삼각형,38475,10,['dp'],[25]
5627,7562,나이트의 이동,22046,10,"['bfs', 'graphs', 'graph_traversal']","[126, 7, 11]"
6476,7562,–댄몄 대,22046,10,"['bfs', 'graphs', 'graph_traversal']","[126, 7, 11]"
9565,10816,숫자 카드 2,36042,7,"['binary_search', 'data_structures', 'hash_set...","[12, 175, 136, 97]"
9617,10870,피보나치 수 5,55086,4,"['implementation', 'math']","[102, 124]"


## test 데이터로

In [143]:
distances, indices = knn.kneighbors([X_test_scaled[0]])

# 필터링된 데이터 프레임 생성
nearest_neighbors_df = X_train.iloc[indices[0]]

In [146]:
df = user.loc[nearest_neighbors_df.index, :]

In [148]:
# 모든 문제의 고유 목록 생성
all_problems = set()
for row in df.itertuples():
    all_problems.update(row.solvedProblemList)
    all_problems.update(row.triedNotsolvedList)
all_problems = sorted(all_problems)

# 이진 상호작용 행렬 생성
interaction_matrix = pd.DataFrame(0, index=df['id'], columns=all_problems)

for row in df.itertuples():
    interaction_matrix.loc[row.id, row.solvedProblemList] = 1

In [ ]:
# 데이터프레임을 PyTorch 텐서로 변환
train_tensor = torch.tensor(interaction_matrix.values).float().to(device)


In [ ]:
# TensorDataset 및 DataLoader 생성
dataset = TensorDataset(train_tensor)
data_loader = DataLoader(dataset, batch_size=5000, shuffle=True)

model = EASE(num_items=train_tensor.shape[1], reg_lambda=100).to(device)

model.fit(data_loader)

In [158]:
solved_problems = user.iloc[X_test.index[0]]['solvedProblemList']

In [159]:
# 문제 번호와 칼럼 인덱스의 매핑 생성
problem_to_index = {problem: idx for idx, problem in enumerate(interaction_matrix.columns)}

In [161]:

# 사용자가 풀었던 문제 번호를 인덱스로 변환
solved_problems_indices = [problem_to_index[problem] for problem in solved_problems if problem in problem_to_index]
# 전체 문제 수
num_problems = train_tensor.shape[1]

# 사용자 상호작용 텐서 생성
user_interaction_tensor = torch.zeros(num_problems)
user_interaction_tensor[solved_problems_indices] = 1
# 텐서를 모델이 있는 디바이스로 이동
user_interaction_tensor = user_interaction_tensor.to(device)
# 모델을 사용하여 추천 점수 계산
predicted_scores = model(user_interaction_tensor)

# 이미 풀었던 문제의 점수를 제외
predicted_scores[solved_problems_indices] = -np.inf

# 추천 점수 기반으로 추천 문제 선택 (예: 상위 N개 문제)
recommended_problems = torch.topk(predicted_scores, k=10).indices


In [162]:
# 칼럼 인덱스에서 문제 번호로의 역매핑 생성 -> 위에있는 값이 value라서 key로 찾을 수 없음 그래서 역매핑해야됨
index_to_problem = {idx: problem for problem, idx in problem_to_index.items()}
# 추천된 인덱스를 문제 번호로 변환
recommended_problem_numbers = [index_to_problem[idx.item()] for idx in recommended_problems]
recommended_problem_numbers =list(map(int, recommended_problem_numbers))
problem[problem['problemId'].isin(recommended_problem_numbers)]

,problemId,title,acceptedUserCount,level,key,bojTagId
10,1010,다리 놓기,34577,6,"['combinatorics', 'dp', 'math']","[6, 25, 124]"
18,1018,체스판 다시 칠하기,43928,7,['bruteforcing'],[125]
157,1157,단어 공부,83256,5,"['implementation', 'string']","[102, 158]"
754,1759,암호 만들기,22908,11,"['backtracking', 'bruteforcing', 'combinatoric...","[5, 125, 6, 124]"
905,1912,연속합,36385,9,['dp'],[25]
1648,2644,촌수계산,18432,9,"['graphs', 'graph_traversal', 'bfs', 'dfs']","[7, 11, 126, 127]"
3912,4963,섬의 개수,23781,9,"['graphs', 'graph_traversal', 'bfs', 'dfs']","[7, 11, 126, 127]"
4718,6603,로또,22184,9,"['backtracking', 'combinatorics', 'math', 'rec...","[5, 6, 124, 62]"
8805,10026,적록색약,26424,11,"['bfs', 'dfs', 'graphs', 'graph_traversal']","[126, 127, 7, 11]"
9617,10870,피보나치 수 5,55086,4,"['implementation', 'math']","[102, 124]"
